In [266]:
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
import yfinance as yf

In [267]:
tsla = yf.Ticker("tsla")

tsla_hist = tsla.history(period='5y', interval='1d', end='2025-06-13')
# tsla_hist

In [268]:
tsla_hist.drop(columns=['Dividends', 'Stock Splits'], inplace=True)

In [269]:
tsla_hist.index =pd.to_numeric(tsla_hist.index)
tsla_hist.index = tsla_hist.index/(max(tsla_hist.index)) + 1

In [270]:
for col in tsla_hist.columns:
    tsla_hist[col] = tsla_hist[col]/max(tsla_hist[col]) + 1

In [271]:
X = tsla_hist.drop(columns=['Close'])
y = tsla_hist['Close']

In [272]:
X['Date'] = X.index

In [273]:
for delay in range(1,4):
    X[f"Delay {delay}"] = y.iloc[(3-delay):-(delay)]

In [274]:
X_sample = [[d, h, l, o, vol, v1, v2, v3] for d, h, l, o, vol, v1, v2, v3 in zip(X['Date'].iloc[3:].values, X['High'].iloc[3:].values, X['Low'].iloc[3:].values,X['Open'].iloc[3:].values, X['Volume'].iloc[3:].values, X['Delay 1'].dropna().values, X['Delay 2'].dropna().values, X['Delay 3'].dropna().values)]
y_sample = y.iloc[3:].values

In [275]:
len(X['Date'].iloc[3:].values), len(X_sample)

(1252, 1252)

In [276]:
goods = []
for samples in X_sample:
    for sample in samples:
        if np.isnan(sample):
            print("contains nan")
        elif sample ==0:
            print("contains 0")
        elif sample != None:
            goods.append(sample)

In [277]:
len(goods)

10016

In [278]:
len(y_sample), len(X_sample)

(1252, 1252)

In [279]:
torch.set_default_dtype(torch.float64)

In [280]:
X = torch.from_numpy(np.array(X))
y = torch.from_numpy(np.array(y))

In [281]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [282]:
len(X_test)

251

In [283]:
train_data = [(X,y) for X, y in zip(X[:-273], y[:-273])]
test_data = [(X,y) for X, y in zip(X[-273:], y[-273:])]

In [284]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloaader= DataLoader(test_data, batch_size=32, shuffle=True)

In [285]:
test_trues = []

for (X, y) in train_dataloader:
    # if np.isnan(X):
    #     test_trues.append(X)
    # if np.isnan(y):
    #     test_trues.append(y)
    print(X)

tensor([[1.5234, 1.5167, 1.5290, 1.3000, 1.9635, 1.5093, 1.5093, 1.5093],
        [1.3788, 1.3790, 1.3771, 1.2551, 1.9590, 1.3604, 1.3604, 1.3604],
        [1.5130, 1.5408, 1.5291, 1.1403, 1.9452, 1.5384, 1.5384, 1.5384],
        [1.6577, 1.6452, 1.6249, 1.1388, 1.9438, 1.6066, 1.6066, 1.6066],
        [1.5214, 1.5360, 1.5412, 1.1947, 1.9693, 1.5442, 1.5442, 1.5442],
        [1.3055, 1.3133, 1.3122, 1.5333, 1.9134, 1.3110, 1.3110, 1.3110],
        [1.7667, 1.7527, 1.7844, 1.0735, 1.9420, 1.7486, 1.7486, 1.7486],
        [1.3058, 1.3152, 1.3138, 1.4380, 1.9143, 1.3124, 1.3124, 1.3124],
        [1.2028, 1.1989, 1.2019, 1.1693, 1.9125, 1.1971, 1.1971, 1.1971],
        [1.6015, 1.5864, 1.6100, 1.1155, 1.9206, 1.5908, 1.5908, 1.5908],
        [1.5092, 1.5219, 1.5271, 1.1528, 1.9469, 1.5226, 1.5226, 1.5226],
        [1.4198, 1.4175, 1.4265, 1.2262, 1.9631, 1.4250, 1.4250, 1.4250],
        [1.5043, 1.5067, 1.5216, 1.1757, 1.9690, 1.5134, 1.5134, 1.5134],
        [1.3959, 1.4071, 1.4034, 1.366

In [286]:
class MLP(nn.Module):
    def __init__(self, input_features, hidden_features, output_features):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(input_features, hidden_features),
            nn.ReLU()
        )
        self.l2 = nn.Sequential(
            nn.Linear(hidden_features, output_features),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        return x

In [287]:
mlpmodel = MLP(8, 15, 1)

In [288]:
loss_fn = nn.MSELoss()

optimizer = torch.optim.SGD(params=mlpmodel.parameters(), lr=0.0001)

In [289]:
EPOCHS = 1

mlpmodel.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch, (X, y) in enumerate(train_dataloader):
        preds = mlpmodel(X)
        # print(f"Preds + {preds}")
        loss = loss_fn(torch.unsqueeze(y, 1), preds)
        total_loss+= loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"We're on batch: {batch} + Loss: {total_loss/batch}")


We're on batch: 0 + Loss: inf
We're on batch: 1 + Loss: 2.7235574924143138
We're on batch: 2 + Loss: nan
We're on batch: 3 + Loss: nan
We're on batch: 4 + Loss: nan
We're on batch: 5 + Loss: nan
We're on batch: 6 + Loss: nan
We're on batch: 7 + Loss: nan
We're on batch: 8 + Loss: nan
We're on batch: 9 + Loss: nan
We're on batch: 10 + Loss: nan
We're on batch: 11 + Loss: nan
We're on batch: 12 + Loss: nan
We're on batch: 13 + Loss: nan
We're on batch: 14 + Loss: nan
We're on batch: 15 + Loss: nan
We're on batch: 16 + Loss: nan
We're on batch: 17 + Loss: nan
We're on batch: 18 + Loss: nan
We're on batch: 19 + Loss: nan
We're on batch: 20 + Loss: nan
We're on batch: 21 + Loss: nan
We're on batch: 22 + Loss: nan
We're on batch: 23 + Loss: nan
We're on batch: 24 + Loss: nan
We're on batch: 25 + Loss: nan
We're on batch: 26 + Loss: nan
We're on batch: 27 + Loss: nan
We're on batch: 28 + Loss: nan
We're on batch: 29 + Loss: nan
We're on batch: 30 + Loss: nan
